In [1]:
'''
训练Hybrid VAE，预测 28天死亡率及院内死亡率
'''
import os, sys
from datetime import datetime
import argparse
import pandas as pd
import json
import pickle
import joblib
import torch
from torchinfo import summary
import optuna
from optuna.visualization import (
    plot_parallel_coordinate,
    plot_param_importances,
    plot_contour,
    plot_slice,
    plot_optimization_history,
    plot_pareto_front,
)
import seaborn as sns
import plotly.graph_objects as go
sns.set_theme('paper')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold 

# 检测运行环境
IN_NOTEBOOK = None
def in_notebook():
    return 'IPKernelApp' in getattr(globals().get('get_ipython', lambda: None)(), 'config', {})
    
if in_notebook():
    IN_NOTEBOOK = True
    print('run in notebook')
    pd.set_option('display.max_columns', None)
    from IPython.display import clear_output, display
    notebook_dir = os.getcwd()
    src_path = os.path.abspath(os.path.join(notebook_dir, '..'))
    RUN_MODE = 'tuning' # reload: 重现study; tuning 搜索超参数
    N_TRIAL = 100
else:
    IN_NOTEBOOK = False
    print('run in script')
    src_path = os.path.abspath(os.path.dirname(os.path.dirname(__file__)))
    parser = argparse.ArgumentParser(description='')
    parser.add_argument('-n', metavar= 50, type=int, default=50,help='''optuna优化尝试次数''')
    sys_args = parser.parse_args()
    N_TRIAL = sys_args.n
    RUN_MODE = 'tuning' # 脚本模式只做tuing!

sys.path.append(src_path) if src_path not in sys.path else None
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from src.utils import *
from src.model_utils import to_numpy, check_tensor
from src.metrix import cal_ci, format_ci
from src.setup import *
from risk_setup import *
from risk_model.HybridVAE import HybridVAEMultiTaskSklearn

set_random_seed(19960816)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'current device: {DEVICE}')
risk_hybrid_vae = f'{MODELS}/risk_models/Hybrid_VAE_model/'
os.makedirs(risk_hybrid_vae, exist_ok=True)

run in notebook
current device: cuda


In [2]:
df = pd.read_csv(f'{DATA}/imputed/MIMIC_IV_clean_imputed.tsv.gz', sep='\t', index_col='ID')
df = df.sample(frac=0.3) # 快速尝试 确定调参范围
features, _, _, outcomes = get_risk_model_features()
X, y = load_data(df, outcome_ix=0) # 这里加载了 28-d mortality 作为预测目标

# load multi-task y
y = df[outcomes].copy() 

# standardization of X
std_processor = StandardScaler()
X_array = std_processor.fit_transform(X)
X = pd.DataFrame(X_array, index=X.index, columns=X.columns)
joblib.dump(std_processor, f'{risk_hybrid_vae}/MIMIC_StandardScaler.joblib')

print(f'training data: {X.shape}')

training data: (6003, 31)


# Usage Example

In [3]:
# if IN_NOTEBOOK:
#     kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=19960816)
#     aucs_1 = []
#     aucs_2 = []
#     for i, (train_index, val_index) in enumerate(kf.split(X, y.iloc[:,0])):
#         # 划分训练集和验证集
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model = HybridVAEMultiTaskSklearn(input_dim=X_train.shape[1],
#                                         task_count=y_train.shape[1],
#                                         layer_strategy='l',
#                                         vae_hidden_dim=50,
#                                         vae_depth=1,
#                                         vae_dropout_rate=0.3,
#                                         latent_dim=10,
#                                         predictor_hidden_dim=20,
#                                         predictor_depth=1,
#                                         task_hidden_dim=20,
#                                         task_depth=1,
#                                         predictor_dropout_rate=0.3,
#                                         vae_lr=1e-3,
#                                         vae_weight_decay=1e-3,
#                                         multitask_lr=1e-3,
#                                         multitask_weight_decay=1e-3,
#                                         alphas=None,
#                                         beta=1.0,
#                                         gamma_task=10000.0,
#                                         batch_size=500,
#                                         validation_split=0.2,
#                                         use_lr_scheduler=True,
#                                         lr_scheduler_factor=0.2,
#                                         lr_scheduler_patience=25,
#                                         use_batch_norm=True,
#                                         )

#         model = model.fit(X_train, y_train,
#                         epochs=2000, 
#                         early_stopping=True, 
#                         patience=100,
#                         verbose=2, 
#                         animate_monitor=True,
#                         plot_path=None,
#                         save_weights_path=None
#                         )

#         auc1, auc2 = model.score(X_val, y_val)
#         aucs_1.append(auc1)
#         aucs_2.append(auc2)
        
#     # assume t-distribution for 95% CI calculation
#     mean_auc_1, lower_1, upper_1 = cal_ci(aucs_1, alpha=0.05, method='t')
#     mean_auc_2, lower_2, upper_2 = cal_ci(aucs_2, alpha=0.05, method='t')
#     print(f'AUC of {outcomes[0]}: {format_ci(mean_auc_1, lower_1, upper_1, 3)}')
#     print(f'AUC of {outcomes[1]}: {format_ci(mean_auc_2, lower_2, upper_2, 3)}')
    
#     total_loss, recon_loss, kl_loss, task_loss = model.eval_loss(X_val, y_val)
#     print(f'Losses on last fold: {total_loss=:.4f}, {recon_loss=:.4f}, {kl_loss=:.4f}, {task_loss=:.4f}')
#     print(summary(model, input_size=(model.batch_size, X.shape[1])))

# Hyper Parameter Tuning

In [ ]:
if RUN_MODE=='tuning':

    def objective(trial):
        try:
            # 定义需要调优的超参数范围
            # network structure
            layer_strategy = trial.suggest_categorical("layer_strategy", ['c', 'l', 'g'])
            vae_hidden_dim = trial.suggest_int("vae_hidden_dim", 20, 200, step=10)
            vae_depth = trial.suggest_int("vae_depth", 0, 3, step=1)
            vae_dropout_rate = trial.suggest_float("vae_dropout_rate", 0.05, 0.5, step=0.05)
            latent_dim = trial.suggest_int("latent_dim", 5, 15, step=1)
            predictor_hidden_dim = trial.suggest_int("predictor_hidden_dim", 10, 100, step=10)
            predictor_depth = trial.suggest_int("predictor_depth", 0, 2, step=1)
            task_hidden_dim =  trial.suggest_int("task_hidden_dim", 10, 30, step=5)
            task_depth = trial.suggest_int("task_depth", 0, 2, step=1)
            predictor_dropout_rate = trial.suggest_float("predictor_dropout_rate", 0.05, 0.5, step=0.05)
            # learning and normalization
            vae_lr = trial.suggest_float("vae_lr", 1e-5, 1e-3, log=True)
            vae_weight_decay = trial.suggest_float("vae_weight_decay", 1e-5, 1e-2, log=True)
            multitask_lr = trial.suggest_float("multitask_lr", 1e-5, 1e-3, log=True)
            multitask_weight_decay = trial.suggest_float("multitask_weight_decay", 1e-5, 1e-2, log=True)
            beta = trial.suggest_float("beta", 1.0, 5.0, step=0.1)
            gamma_task = trial.suggest_float("gamma_task", 1.0, 100000, log=True) 
            batch_size = trial.suggest_int("batch_size", 200, 1000, step=100)
            validation_split = trial.suggest_categorical("validation_split", [0.2, 0.3, 0.4])
            use_lr_scheduler = trial.suggest_categorical("use_lr_scheduler", [True, False])
            lr_scheduler_factor = trial.suggest_categorical("lr_scheduler_factor", [0.1, 0.2])
            fit_patience = 100
            lr_scheduler_patience = trial.suggest_categorical("lr_scheduler_patience", [int(fit_patience/2), int(fit_patience/3), int(fit_patience/4),])
            use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False])
            # 初始化模型
            model = HybridVAEMultiTaskSklearn(input_dim=X.shape[1],
                                            task_count=y.shape[1],
                                            layer_strategy=layer_strategy,
                                            vae_hidden_dim=vae_hidden_dim,
                                            vae_depth=vae_depth,
                                            vae_dropout_rate=vae_dropout_rate,
                                            latent_dim=latent_dim,
                                            predictor_hidden_dim=predictor_hidden_dim,
                                            predictor_depth=predictor_depth,
                                            task_hidden_dim=task_hidden_dim,
                                            task_depth=task_depth,
                                            predictor_dropout_rate=predictor_dropout_rate,
                                            vae_lr=vae_lr,
                                            vae_weight_decay=vae_weight_decay,
                                            multitask_lr=multitask_lr,
                                            multitask_weight_decay=multitask_weight_decay,
                                            alphas=None,
                                            beta=beta,
                                            gamma_task=gamma_task,
                                            batch_size=batch_size,
                                            validation_split=validation_split,
                                            use_lr_scheduler=use_lr_scheduler,
                                            lr_scheduler_factor=lr_scheduler_factor,
                                            lr_scheduler_patience=lr_scheduler_patience,
                                            use_batch_norm=use_batch_norm,
                                            )
            # 实现交叉验证
            kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=19960816)
            aucs_1 = []
            aucs_2 = []
            recon_losses = []
            vae_losses = []

            for i, (train_index, val_index) in enumerate(kf.split(X, y.iloc[:,0])): # stratified by primary outcome
                # 划分训练集和验证集
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                
                # 训练模型
                model.fit(X_train, y_train,
                            epochs=2000, 
                            early_stopping=True, 
                            patience=fit_patience,
                            verbose=IN_NOTEBOOK, 
                            animate_monitor=False,
                            plot_path=None,
                            save_weights_path=None)
                
                # 验证模型并记录分数
                auc1, auc2 = model.score(X_val, y_val)  # 默认AUC
                total_loss, recon_loss, kl_loss, task_loss = model.eval_loss(X_val, y_val)
                vae_loss = recon_loss + model.beta * kl_loss

                aucs_1.append(auc1)
                aucs_2.append(auc2)
                recon_losses.append(recon_loss)
                vae_losses.append(vae_loss)
                
                # 性能太差的提前终止
                if np.mean(aucs_1) <= 0.7 and np.mean(aucs_2) <= 0.7:
                    break
            
            # 返回平均交叉验证分数（负均方误差）
            return np.mean(aucs_1), np.mean(aucs_2), np.mean(vae_losses), np.mean(recon_losses)
        
        except Exception as e:
            print(f"Trial failed with error: {e}")
            return -np.inf, -np.inf, np.inf, np.inf

    # 日志功能：设置 Optuna 的日志级别
    # optuna.logging.set_verbosity(optuna.logging.INFO)

    # 实时打印当前最佳结果
    def trial_callback(study, trial):
        is_multiobjective = len(study.directions) > 1  # 判断是否多目标优化

        if is_multiobjective:
            pareto_front = study.best_trials  # 获取 Pareto 前沿解

            if IN_NOTEBOOK:
                clear_output(wait=True)  # 清除之前的输出
                df_trials = study.trials_dataframe()  # 获取当前的试验数据
                display(df_trials)  # 动态显示最新的 dataframe

                # 打印 Pareto 前沿解的信息
                print(f"Number of Pareto optimal solutions: {len(pareto_front)}")
                for i, pareto_trial in enumerate(pareto_front):
                    print(f"Pareto solution {i}: Values {pareto_trial.values}, Params {pareto_trial.params}")
            else:
                # best_trial = pareto_front[0]  # 选择首选 Pareto 前沿解
                # print(
                #     f"Trial {trial.number}/{N_TRIAL} finished with value: {trial.values} and parameters: {trial.params} | "
                #     f"Current best value: {best_trial.values} and parameters: {best_trial.params} | "
                #     f"Number of Pareto optimal solutions: {len(pareto_front)}"
                # )
                pass
        else:
            if IN_NOTEBOOK:
                clear_output(wait=True)  # 清除之前的输出
                df_trials = study.trials_dataframe()  # 获取当前的试验数据
                display(df_trials)  # 动态显示最新的 dataframe

                # 打印当前最优解的信息
                print(f"Current best value: {study.best_value}")
                print(f"Current best parameters: {study.best_params}")
                print(f"Current best trial: {study.best_trials}")
            else:
                # print(
                #     f"Trial {trial.number}/{N_TRIAL} finished with value: {trial.value} and parameters: {trial.params} | "
                #     f"Current best value: {study.best_value} and parameters: {study.best_params}"
                # )
                pass
                
    # 使用 Optuna 优化
    study = optuna.create_study(directions=["maximize", "maximize", "minimize", 'minimize'], sampler=optuna.samplers.NSGAIISampler(seed=19960816))  # 或 "minimize"，取决于评分标准
    study.optimize(objective, n_trials=N_TRIAL, callbacks=[trial_callback])

    # 保存实验结果
    with open(f"{risk_hybrid_vae}/optuna_study.pkl", "wb") as f:
        print('调参结束，正在保存optuna调参试验结果')
        pickle.dump(study, f)

    # 获取 Pareto 前沿解
    pareto_front = study.best_trials
    print("Pareto Front Solutions:")
    for trial in pareto_front:
        print(f"Trial {trial.number}: Values {trial.values}, Params {trial.params}")

    # 保存 Pareto 解到文件
    pareto_data = [
        {"trial_number": trial.number, "values": trial.values, "params": trial.params}
        for trial in pareto_front
    ]
    with open(f"{risk_hybrid_vae}/pareto_solutions.json", "w") as f:
        json.dump(pareto_data, f)


    # 保存完整调参历史为 xlsx 文件
    df_trials = study.trials_dataframe()
    df_trials.to_excel(f"{risk_hybrid_vae}/tuning_history.xlsx", index=False)

,number,values_0,values_1,values_2,values_3,datetime_start,datetime_complete,duration,params_batch_size,params_beta,params_gamma_task,params_latent_dim,params_layer_strategy,params_lr_scheduler_factor,params_lr_scheduler_patience,params_multitask_lr,params_multitask_weight_decay,params_predictor_depth,params_predictor_dropout_rate,params_predictor_hidden_dim,params_task_depth,params_task_hidden_dim,params_use_batch_norm,params_use_lr_scheduler,params_vae_depth,params_vae_dropout_rate,params_vae_hidden_dim,params_vae_lr,params_vae_weight_decay,params_validation_split,system_attrs_nsga2:generation,state
0,0,-inf,-inf,inf,inf,2024-12-09 01:33:31.175660,2024-12-09 01:33:33.288192,0 days 00:00:02.112532,700,2.0,24745.558871,11,g,0.2,50,0.000016,0.000431,0,0.30,50,0,25,False,False,2,0.25,140,0.000337,0.001163,0.3,0,COMPLETE
1,1,0.767915,0.767646,52.587955,14.734551,2024-12-09 01:33:33.315908,2024-12-09 01:50:09.847051,0 days 00:16:36.531143,200,3.8,59899.844052,13,c,0.2,50,0.000029,0.002568,0,0.10,80,1,10,True,False,0,0.20,180,0.000042,0.001905,0.2,0,COMPLETE
2,2,0.694394,0.696123,32.162899,22.614807,2024-12-09 01:50:09.872325,2024-12-09 01:51:08.694453,0 days 00:00:58.822128,700,3.6,2428.114596,11,c,0.2,50,0.000073,0.001514,1,0.40,30,2,30,True,True,0,0.40,40,0.000914,0.002075,0.4,0,COMPLETE
3,3,0.462328,0.436206,32.332090,31.855052,2024-12-09 01:51:08.720808,2024-12-09 01:52:15.466702,0 days 00:01:06.745894,500,3.1,34677.366712,6,c,0.2,25,0.000014,0.000068,2,0.20,20,1,15,False,True,3,0.20,80,0.000033,0.000029,0.3,0,COMPLETE
4,4,-inf,-inf,inf,inf,2024-12-09 01:52:15.494170,2024-12-09 01:52:15.531088,0 days 00:00:00.036918,300,3.8,211.683980,12,c,0.1,50,0.000265,0.005231,0,0.45,90,0,10,False,False,3,0.35,120,0.000016,0.000042,0.4,0,COMPLETE
5,5,0.453758,0.440545,36.200991,31.042419,2024-12-09 01:52:15.556744,2024-12-09 01:52:23.638225,0 days 00:00:08.081481,900,1.7,35.470639,15,c,0.1,50,0.000030,0.005383,2,0.45,90,1,20,True,True,3,0.10,170,0.000188,0.000968,0.3,0,COMPLETE
6,6,0.738118,0.745652,33.167007,25.986757,2024-12-09 01:52:23.665277,2024-12-09 02:01:33.410323,0 days 00:09:09.745046,900,3.7,14260.790869,8,g,0.1,50,0.000307,0.000638,0,0.35,70,2,20,False,False,1,0.05,80,0.000018,0.000526,0.4,0,COMPLETE
7,7,0.764057,0.772157,34.114483,21.419504,2024-12-09 02:01:33.439815,2024-12-09 02:19:28.677835,0 days 00:17:55.238020,300,4.5,15581.362303,11,c,0.1,50,0.000216,0.005179,1,0.15,40,1,10,True,False,1,0.50,190,0.000130,0.000218,0.2,0,COMPLETE
8,8,0.711676,0.716669,24.567729,17.855014,2024-12-09 02:19:28.708784,2024-12-09 02:35:58.022681,0 days 00:16:29.313897,300,1.6,16.295643,15,c,0.1,33,0.000060,0.003343,0,0.50,10,1,15,True,False,2,0.20,160,0.000512,0.005170,0.4,0,COMPLETE
9,9,-inf,-inf,inf,inf,2024-12-09 02:35:58.054199,2024-12-09 02:35:58.105764,0 days 00:00:00.051565,900,3.9,125.157671,6,l,0.2,50,0.000474,0.000014,1,0.10,40,0,10,False,True,2,0.35,20,0.000029,0.000027,0.3,0,COMPLETE


Number of Pareto optimal solutions: 12
Pareto solution 0: Values [0.7679149297778617, 0.7676459266133824, 52.58795513564, 14.734550794259992], Params {'layer_strategy': 'c', 'vae_hidden_dim': 180, 'vae_depth': 0, 'vae_dropout_rate': 0.2, 'latent_dim': 13, 'predictor_hidden_dim': 80, 'predictor_depth': 0, 'task_hidden_dim': 10, 'task_depth': 1, 'predictor_dropout_rate': 0.1, 'vae_lr': 4.184147539029173e-05, 'vae_weight_decay': 0.001904990804866105, 'multitask_lr': 2.9172622587607467e-05, 'multitask_weight_decay': 0.0025680970563357617, 'beta': 3.8000000000000003, 'gamma_task': 59899.84405185852, 'batch_size': 200, 'validation_split': 0.2, 'use_lr_scheduler': False, 'lr_scheduler_factor': 0.2, 'lr_scheduler_patience': 50, 'use_batch_norm': True}
Pareto solution 1: Values [0.7640571608094268, 0.7721570343159858, 34.11448313889833, 21.419504109531122], Params {'layer_strategy': 'c', 'vae_hidden_dim': 190, 'vae_depth': 1, 'vae_dropout_rate': 0.5, 'latent_dim': 11, 'predictor_hidden_dim': 40

Training Progress:  32%|█████▋            | 632/2000 [00:41<01:29, 15.24epoch/s, Train VAE Loss=31.1216, Val VAE Loss=30.4007, Train AUC=0.5101, Val AUC=0.4093]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Training Progress:  59%|██████████       | 1184/2000 [02:24<01:39,  8.21epoch/s, Train VAE Loss=20.6835, Val VAE Loss=20.0676, Train AUC=0.7525, Val AUC=0.7110]


Early stopping triggered due to no improvement in both VAE and task losses.


Training Progress:  23%|████              | 457/2000 [00:55<03:06,  8.25epoch/s, Train VAE Loss=21.6749, Val VAE Loss=20.8313, Train AUC=0.7256, Val AUC=0.7176]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Training Progress:  53%|█████████        | 1064/2000 [01:06<00:58, 16.07epoch/s, Train VAE Loss=26.1738, Val VAE Loss=24.3584, Train AUC=0.6505, Val AUC=0.7288]


Early stopping triggered due to no improvement in both VAE and task losses.


Training Progress:  16%|██▊               | 318/2000 [00:19<01:44, 16.10epoch/s, Train VAE Loss=27.8733, Val VAE Loss=26.4069, Train AUC=0.5757, Val AUC=0.7008]

# Training with best params

In [ ]:
if RUN_MODE=='tuning':
    pass

elif RUN_MODE=='reload':
    with open(f"{risk_hybrid_vae}/optuna_study.pkl", "rb") as f:
        print('正在加载指定optuna调参试验结果')
        study = pickle.load(f)
        
# 使用一个 Pareto 最优解重新初始化模型
pareto_front = study.best_trials
best_trial = pareto_front[0] # 选择第一个 Pareto 解
best_params = best_trial.params

# 使用最佳参数重新初始化模型
best_model = HybridVAEMultiTaskSklearn(input_dim=X.shape[1], task_count=y.shape[1], **best_params)

# 训练最佳模型
print('使用最佳参数进行内部验证估计误差')
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=19960816)
aucs_1 = []
aucs_2 = []
for i, (train_index, val_index) in enumerate(kf.split(X, y.iloc[:,0])):
    # 划分训练集和验证集
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = HybridVAEMultiTaskSklearn(input_dim=X_train.shape[1],
                                    task_count=y_train.shape[1],
                                    layer_strategy='l',
                                    vae_hidden_dim=50,
                                    vae_depth=1,
                                    vae_dropout_rate=0.3,
                                    latent_dim=10,
                                    predictor_hidden_dim=20,
                                    predictor_depth=1,
                                    task_hidden_dim=20,
                                    task_depth=1,
                                    predictor_dropout_rate=0.3,
                                    vae_lr=1e-3,
                                    vae_weight_decay=1e-3,
                                    multitask_lr=1e-3,
                                    multitask_weight_decay=1e-3,
                                    alphas=None,
                                    beta=1.0,
                                    gamma_task=10000.0,
                                    batch_size=256,
                                    validation_split=0.2,
                                    use_lr_scheduler=True,
                                    lr_scheduler_factor=0.2,
                                    lr_scheduler_patience=25,
                                    use_batch_norm=True,
                                    )

    model = model.fit(X_train, y_train,
                    epochs=2000, 
                    early_stopping=True, 
                    patience=100,
                    verbose=2, 
                    animate_monitor=True,
                    plot_path=None,
                    save_weights_path=None
                    )

    auc1, auc2 = model.score(X_val, y_val)
    aucs_1.append(auc1)
    aucs_2.append(auc2)

# assume t-distribution for 95% CI calculation
mean_auc_1, lower_1, upper_1 = cal_ci(aucs_1, alpha=0.05, method='t')
mean_auc_2, lower_2, upper_2 = cal_ci(aucs_2, alpha=0.05, method='t')
print(f'AUC of {outcomes[0]}: {format_ci(mean_auc_1, lower_1, upper_1, 3)}')
print(f'AUC of {outcomes[1]}: {format_ci(mean_auc_2, lower_2, upper_2, 3)}')

total_loss, recon_loss, kl_loss, task_loss = model.eval_loss(X_val, y_val)
print(f'Losses on last fold: {total_loss=:.4f}, {recon_loss=:.4f}, {kl_loss=:.4f}, {task_loss=:.4f}')
print(summary(model, input_size=(model.batch_size, X.shape[1])))

print('使用最佳参数在全集上训练模型 （准备外部验证）')
best_model.fit(X, y,
                epochs=2000, 
                early_stopping=True, 
                patience=200,
                verbose=2, 
                animate_monitor=True,
                plot_path=risk_hybrid_vae,
                save_weights_path=risk_hybrid_vae)

print(f'最终AUC: {best_model.score(X, y)}')

In [ ]:
# 绘制不同的图表
target_args_1 = dict(target = lambda t: -t.values[0], target_name="AUC-primary")
target_args_2 = dict(target = lambda t: -t.values[1], target_name="AUC-secondary")
target_args_3 = dict(target = lambda t: -t.values[2], target_name="VAE loss")
target_args_4 = dict(target = lambda t: -t.values[3], target_name="Reconstruction MSE")
targets_args = dict(targets = lambda t: [t.values[0], -t.values[2]], target_names=["AUC", "VAE loss"])

# 并行坐标图
parallel_coordinate_fig = plot_parallel_coordinate(study, **target_args_1)
parallel_coordinate_fig.update_layout(width=800, height=600)
parallel_coordinate_fig.show() if IN_NOTEBOOK else None
parallel_coordinate_fig.write_image(f"{risk_hybrid_vae}/parallel_coordinate_fig.svg", format='svg', scale=2, width=700, height=500) if not IN_NOTEBOOK else None

# 参数重要性图
param_importance_fig = plot_param_importances(study, **target_args_1)
param_importance_fig.update_layout(width=800, height=600)
param_importance_fig.show() if IN_NOTEBOOK else None
param_importance_fig.write_image(f"{risk_hybrid_vae}/param_importance_fig.svg", format='svg', scale=2, width=700, height=500) if not IN_NOTEBOOK else None

# 平行曲面图
contour_fig = plot_contour(study, **target_args_1)
contour_fig.update_layout(width=1200, height=1200)
contour_fig.show() if IN_NOTEBOOK else None
contour_fig.write_image(f"{risk_hybrid_vae}/contour_fig.svg", format='svg', scale=2, width=1200, height=1200) if not IN_NOTEBOOK else None

# 超参数分布图
slice_fig = plot_slice(study, **target_args_1)
slice_fig.show() if IN_NOTEBOOK else None
slice_fig.write_image(f"{risk_hybrid_vae}/slice_fig.svg", format='svg', scale=2, width=2500, height=400) if not IN_NOTEBOOK else None

# 优化历史图
optimization_history_fig = plot_optimization_history(study, **target_args_1)
optimization_history_fig.update_layout(width=700, height=500)
optimization_history_fig.show() if IN_NOTEBOOK else None
optimization_history_fig.write_image(f"{risk_hybrid_vae}/optimization_history_fig.svg", format='svg', scale=2, width=700, height=500) if not IN_NOTEBOOK else None

# Pareto 前沿图（仅适用于多目标优化）
if len(study.directions) > 1:
    pareto_fig = plot_pareto_front(study, **targets_args)
    pareto_fig.update_layout(width=1200, height=800)
    pareto_fig.show() if IN_NOTEBOOK else None
    pareto_fig.write_image(f"{risk_hybrid_vae}/pareto_fig.svg", format='svg', scale=2, width=1200, height=1200) if not IN_NOTEBOOK else None
